In [ ]:
#Instala python pelo Terminal antes de rodar essa célula
#python.exe -m pip install --upgrade pip

#Instalação pelo
!pip install yfinance
# Install the main crewAI package
!pip install crewai

!pip install "numpy<2"

# Install the main crewAI package and the tools package
# that includes a series of helpful tools for your agents
!pip install 'crewai[tools]'

!pip install langchain --user
!pip install langchain-openai
!pip install langchain-community
!pip install -U duckduckgo-search

In [ ]:
import json
import os
# Importação para não dar erro com o numpy
import numpy as np
np.float_ = np.float64

from datetime import datetime

import yfinance as yf

from crewai import Agent, Task, Crew, Process

from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

In [ ]:
#CRIANDO YAHOO FINANCE TOOL
def fetch_stock_price(ticket):
    stock = yf.download("AAPL", start="2023-08-19", end="2024-08-19")
    return stock

yahoo_finance_tool = Tool(
    name = "Yahoo Finance Tool",
    description = "Fetches stock prices for {ticket} from the last year about a specific stock from Yahoo Finance API",
    func= lambda ticket: fetch_stock_price(ticket)
)

In [ ]:
response = yahoo_finance_tool("AAPL")

In [ ]:
#IMPORTANTO OPENAI LLM - GPT
os.environ["OPENAI_API_KEY"] = "sk-pro-dEzUsDoGDrmW5mT92vbny4HMjnuvMD-l0RFqkAxDzeuUH-uZFUj_9EMSfJAT3BlbkFJNrHqbjlDPV9kPmAnG_c9PxrKdIL02DG1u5yF3XkkL0VdqmSNO_VywtmvcA"
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
stockPriceAnalyst = Agent(
    role="Senior stock price Analyst",
    goal="Find the {ticket} stock price and analyses trends",
    backstory="You're a highly experienced in analysing the price of a specific stock and make predictions about its future price.",
    verbose=True,
    llm= llm,
    max_iter= 5,
    memory=True,
    allow_delegation=False,
    tools=[yahoo_finance_tool]
)

In [ ]:
getStockPrice = Task(
    description="Analyze the stock {ticket} price history and create a tren analyses of up, down or sideways",
    expected_output=""""Specify the current trend stock price - up, down or sideways.
    eg. stock='APPL, price UP'
""",
)

In [ ]:
#IMPORTANDO A TOOL DE SEARCH
search_tool = DuckDuckGoSearchResults(backend='news', num_results=10)

In [ ]:
newsAnalyst = Agent(
    role="Stock New Analyst",
    goal="""Create a short summary of the market news related to the stock {ticket} company. Specify the current trend - up, down or sideways with the news context.
    For each request stock asset, specify a number between 0 and 100, where 0 is extreme fear and 100 is extreme greed""",
    backstory="""You're a highly experienced in analysing the market trends and news and have tracked assets for more than 10 years.
    
    You're also master level analysts in the tradicional markets and have deep understaning of human psychology.

    You understand news, theirs titles and information, but you look at those with a health dose os skepticism.
    You consider also the source of the news articles.
    """,
    verbose=True,
    llm= llm,
    max_iter= 5,
    memory=True,
    allow_delegation=False,
    tools=[yahoo_finance_tool]
)

In [ ]:
get_news = Task(
    description="""Take the stock and always include BTC to it (if not request).
    Use the search tool to search on individually.

    The current date is {datetime.now()}.

    Compose the results into a helpful report
    """,
    expected_output=""""A summary of the overall market and one sentence summary for each request asset.
    Include a fear/greed score for each asset based on the news. Use formar:
    <STOCK ASSET>
    <SUMMARY BASED ON NEWS>
    <TREND PREDICTION>
    <FEAR/GREED SCORE>
""",
    agent= newsAnalyst
)

In [ ]:
stockAnalystWrite = Agent(
    role="Senior Stock Analyst Writer",
    goal="""Aanlyze the trends price and news and write an insightfull compeling and informative 3 paragraph long newsletter based on the stock report and price trend.""",
    backstory="""You're widely accepted as the best stock analust in the market. You understand complet concepts and create compelling stories and narratives that resonate with wider audiences.
    You understand macro factors and combine multiple theories - eg. cycle theory and fundamental analyses.
    You're able to hold multuple opinions when analyzing anything""",
    verbose=True,
    llm= llm,
    max_iter= 5,
    memory=True,
    allow_delegation=True
)

In [ ]:
writeAnalyses = Task(
    description="""Use the stock price trend and the stock news report to create an analyses and write the newsletter about the {ticket} company
    that is brief and highlights the most important points.
    Focus on the stock price trend, news and fear;greed score. What are the near future considerations?
    Include the previous analyses of stock trend and news summary.
    """,
    expected_output="""an eloquent 3 paragraphs newsletter formated as markdown in an easy readable manner. It should contain:

    - 3 bullets executive summary
    - Introduction - set the overall picture and spike up the interest
    - main part provides the meat of the analysis including the news summary and fear/greed scores
    - summary - key facts and concrete future trend prediction - up, down and sideways.
    
    """,
    agent= stockAnalystWrite,
    context=[getStockPrice, get_news]
)

In [ ]:
crew = Crew(
    agents = [stockPriceAnalyst, newsAnalyst, stockAnalystWrite],
    tasks = [getStockPrice, get_news, writeAnalyses],
    verbose = True,
    process = Process.hierarchical,
    full_output = True,
    share_crew = False,
    manager_llm = llm,
    max_iter = 15
)

In [ ]:
results= crew.kickoff(inputs={'ticket': 'AAPL'})